Importamos las librerías


In [ ]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import matplotlib.pyplot as plt



Creamos el dataset

In [4]:
df_bosque = pd.read_csv('covtype.data')
data = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']

soil_type = ['Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
                      'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
                      'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15',
                      'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
                      'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25',
                      'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
                      'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
                      'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']
wilderness_area = ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4']

for i in range(1, 5):
    col_nombre = f'Wilderness_Area{i}'
    data.append(col_nombre)
    wilderness_area.append(col_nombre)

for i in range(1, 41):
    col_nombre = f'Soil_Type{i}'
    data.append(col_nombre)
    soil_type.append(col_nombre)

data.append('Cover_Type')
df_bosque.columns = data

df_bosque



,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
1,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
2,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
3,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
4,2579,132,6,300,-15,67,230,237,140,6031,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581006,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581007,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581008,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581009,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3


Juntamos las columnas del suelo y áreas silvestres. Y, a continuación, borramos las columnas que no nos interesan.

In [5]:
# Juntamos las columnas de Wilderness_Area y Soil_Type en una sola columna

df_bosque['Soil_Type'] = df_bosque[soil_type].sum(axis=1)
df_bosque['Wilderness_Area'] = df_bosque[wilderness_area].sum(axis=1)

# Eliminamos las columnas que ya no necesitamos

df_bosque = df_bosque.drop(columns=soil_type)
df_bosque = df_bosque.drop(columns=wilderness_area)

df_bosque

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Soil_Type,Wilderness_Area
0,2590,56,2,212,-6,390,220,235,151,6225,5,2,2
1,2804,139,9,268,65,3180,234,238,135,6121,2,2,2
2,2785,155,18,242,118,3090,238,238,122,6211,2,2,2
3,2595,45,2,153,-1,391,220,234,150,6172,5,2,2
4,2579,132,6,300,-15,67,230,237,140,6031,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581006,2396,153,20,85,17,108,240,237,118,837,3,2,2
581007,2391,152,19,67,12,95,240,237,119,845,3,2,2
581008,2386,159,17,60,7,90,236,241,130,854,3,2,2
581009,2384,170,15,60,5,90,230,245,143,864,3,2,2


Ejercicio 1

Para conseguir un dataset con una dimensión reducidad, aplica la técnica de Selección de variables basada en árbol de decisión mediante las importancias de cada variable (Decision Trees Importances):

- Filtra el tablón para quedarnos solamente con las variables que aglutinan hasta el 95% de la información que se requiere para estimar la variable objetivo.

- random_state=100

In [8]:
d_arbol = ExtraTreesClassifier(random_state=100)

X = df_bosque.drop(['Cover_Type'], axis=1)
y = df_bosque['Cover_Type']

# Entrenamos el clasificador para poder realizar predicciones más precisas

d_arbol.fit(X, y)

# Calculamos la importancia, lo que puede ayudarnos a seleccionar las características más importantes y eliminar las menos importantes.

importancia = d_arbol.feature_importances_

# Ordenamos los índices de manera descendente, para que los valores más importantes estén en la parte superior.

indices = np.argsort(importancia)[::-1]

# Realizamos la suma acumulativa de las importancias ordenadas de manera descendente para luego, aplicarles el 95% de la suma total.

importancia_acumulada = np.cumsum(importancia[indices])

# Seleccionamos las características que representan hasta el 95% de la importancia acumulada.

indices_pedidos = indices[importancia_acumulada <= 0.95]

# Filtramos el DataFrame original con las características seleccionadas.

df_reducir = df_bosque.iloc[:, indices_pedidos].copy()

# Volvemos a añadir la columna Cover_Type al DataFrame.

df_reducir['Cover_Type'] = df_bosque['Cover_Type'].copy()

df_reducir


KeyError: "None of [Int64Index([0, 5, 9, 3, 4, 1, 7, 6], dtype='int64')] are in the [columns]"